In [4]:
import pandas as pd
import numpy as np

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

worksheet = gc.open('CB_JAY').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)

# Convert to a DataFrame and render.
#import pandas as pd


[['Question', 'category', 'sentence_code'], ['What is your name', 'profile', '1'], ["What's your name", 'profile', '1'], ['Your good name please', 'profile', '1'], ['What is your good name', 'profile', '1'], ["What's your good name", 'profile', '1'], ['Who are you', 'profile', '1'], ['Who you are', 'profile', '1'], ['Who you ', 'profile', '1'], ['What is your identity', 'profile', '2'], ["What's your identity", 'profile', '2'], ['What are you', 'profile', '2'], ['What you are ', 'profile', '2'], ['What are you actually', 'profile', '2'], ['What you actually are', 'profile', '2'], ['Of what type you are', 'profile', '2'], ['Of what kind you are ', 'profile', '2'], ['you are of what type', 'profile', '2'], ['you are of what kind', 'profile', '2'], ['What is your gender ', 'profile', '3'], ["What's your gender", 'profile', '3'], ['Your gender', 'profile', '3'], ['Who made you ', 'profile', '4'], ['Who created you', 'profile', '4'], ['By whom you are created', 'profile', '4'], ['Who is you

In [18]:
df = pd.DataFrame.from_records(rows)
df.rename(columns=df.iloc[0], inplace=True)
df = df[1:]
df.reset_index(drop=True, inplace=True)
df.head(10)

,Question,category,sentence_code
0,What is your name,profile,1
1,What's your name,profile,1
2,Your good name please,profile,1
3,What is your good name,profile,1
4,What's your good name,profile,1
5,Who are you,profile,1
6,Who you are,profile,1
7,Who you,profile,1
8,What is your identity,profile,2
9,What's your identity,profile,2


In [7]:
df.drop('category', axis=1, inplace=True)

In [8]:
import re
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [10]:
corpus = []

In [11]:
for i in range(0, len(df)):
  review = re.sub('[^a-zA-Z]', ' ', df['Question'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review]
  review = ' '.join(review)
  corpus.append(review)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
x_train = cv.fit_transform(corpus).toarray()

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfIdf = TfidfVectorizer(norm='l2')
x_train = tfIdf.fit_transform(corpus).toarray()

In [14]:
#x_train

In [19]:
y_train = df['category']+" "+df['sentence_code']

In [20]:
y_train

0             profile 1
1             profile 1
2             profile 1
3             profile 1
4             profile 1
             ...       
201             task 20
202             task 20
203    sense of humor 1
204    sense of humor 1
205    sense of humor 1
Length: 206, dtype: object

In [21]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=0.1)
model.fit(x_train, y_train)

MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)

In [22]:
y = model.predict(x_train)

In [23]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_train)
#cm

In [24]:
from sklearn.metrics import accuracy_score
accuracy_score(y_train, y)

1.0

In [25]:
ip = "i want to write a program so open vs code"

In [26]:
test = []
review = re.sub('[^a-zA-Z]', ' ', ip)
review = review.lower()
review = review.split()
review = [ps.stem(word) for word in review]
review = ' '.join(review)
test.append(review)

In [27]:
x = tfIdf.transform(test).toarray()

In [28]:
model.predict(x)

array(['task 6'], dtype='<U16')

In [29]:
import pickle

In [33]:
# save the model to disk
filename = "/content/drive/My Drive/Emma/NLP/NLU/word2vec.sav"
pickle.dump(tfIdf, open(filename, 'wb'))

In [34]:
# save the model to disk
filename = "/content/drive/My Drive/Emma/NLP/NLU/nlu.sav"
pickle.dump(model, open(filename, 'wb'))